In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/My\ Drive/nlp/

/content/gdrive/My Drive/nlp


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
import random

lemmatizer = WordNetLemmatizer()

words=[]
classes = []
documents = []
ignore_words = ['?', '!']

# Загружаем датасет
dataset = pd.read_csv('training_data.csv')[['complaint_text', 'Topic']]
dataset['complaint_text'] = dataset['complaint_text'].astype(str)

selected_data = pd.DataFrame()
for category in dataset['Topic'].unique():
    category_data = dataset[dataset['Topic'] == category].head(500)
    selected_data = pd.concat([selected_data, category_data])

dataset = selected_data


for index, item in dataset.iterrows():
  # take each word and tokenize it
  w = nltk.word_tokenize(item['complaint_text'])
  words.extend(w)
  # adding documents
  documents.append((w, item['Topic']))

  # adding classes to our class list
  if item['Topic'] not in classes:
      classes.append(item['Topic'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('sequential_model.h5', hist)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2500 documents
5 classes [0, 1, 2, 3, 4]
10858 unique lemmatized words ['a', 'aa', 'aacount', 'aarp', 'ab', 'aba', 'aback', 'abandoned', 'abate', 'abe', 'aberration', 'abhorrent', 'abide', 'abided', 'abiding', 'abiilities', 'ability', 'able', 'abnormally', 'abou', 'about', 'above', 'abovementioned', 'abovenamed', 'abovereferenced', 'aboveyou', 'abrasive', 'abreast', 'abroad', 'abrupt', 'abruptly', 'abscond', 'absconding', 'absent', 'absolute', 'absolutely', 'absolve', 'absolved', 'absorb', 'absorbed', 'absurd', 'absurdly', 'abuse', 'abused', 'abusive', 'abusiveness', 'ac', 'acc', 'acccess', 'acccount', 'acccounts', 'accelerate', 'accelerated', 'acceleration', 'accent', 'acceppted', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessible', 'accessing', 'accessory', 'accident', 'accidental', 'accidentally', 'accocated', 'accommadation', 'accommodate', 'accommodation', 'accomodations', 'accompany', 'accomplice', 'accomplish', 'accomplish

<ipython-input-3-62ceb192ee7e>:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Training data created


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
500/500 [==============================] - 5s 8ms/step - loss: 1.3944 - accuracy: 0.4100
Epoch 2/200
500/500 [==============================] - 4s 7ms/step - loss: 1.0843 - accuracy: 0.5832
Epoch 3/200
500/500 [==============================] - 4s 8ms/step - loss: 0.9136 - accuracy: 0.6892
Epoch 4/200
500/500 [==============================] - 5s 10ms/step - loss: 0.8145 - accuracy: 0.7176
Epoch 5/200
500/500 [==============================] - 4s 8ms/step - loss: 0.7273 - accuracy: 0.7504
Epoch 6/200
500/500 [==============================] - 4s 8ms/step - loss: 0.6599 - accuracy: 0.7796
Epoch 7/200
500/500 [==============================] - 4s 8ms/step - loss: 0.6035 - accuracy: 0.7984
Epoch 8/200
500/500 [==============================] - 5s 10ms/step - loss: 0.5690 - accuracy: 0.8068
Epoch 9/200
500/500 [==============================] - 4s 8ms/step - loss: 0.5041 - accuracy: 0.8208
Epoch 10/200
500/500 [==============================] - 4s 7ms/step - loss: 0.4570 - accu